# JobAssistAI: Conversational Jobs Recommendation Chatbot

In [1]:
# Import the libraries
import pandas as pd
import os, json, ast
import openai
from tenacity import retry, wait_random_exponential, stop_after_attempt

## Loading Dataset

In [2]:
# Read the dataset and read the Jobs Dataset
job_df = pd.read_csv('job_postings.csv')
job_df.head()

,Company Name,Job Title,Job Description,Job Location,Job Posting URL,Normalized Salary
0,Moderna,Medical Records Technician,We are looking for a Medical Records Technicia...,"Milwaukee, WI",https://www.linkedin.com/jobs/view/934417/?trk...,68000
1,Amgen,Pharma Quality Manager,We are looking for a Pharma Quality Manager wi...,"Louisville, KY",https://www.linkedin.com/jobs/view/984201/?trk...,100000
2,Deutsch,Director of Marketing,We are looking for a Director of Marketing wit...,"Denver, CO",https://www.linkedin.com/jobs/view/950317/?trk...,120000
3,Bechtel,Junior Civil Engineer,We are looking for a Junior Civil Engineer wit...,"Philadelphia, PA",https://www.linkedin.com/jobs/view/911385/?trk...,62000
4,HDR,Junior Civil Engineer,We are looking for a Junior Civil Engineer wit...,"Louisville, KY",https://www.linkedin.com/jobs/view/988966/?trk...,54000


In [3]:
job_df.isnull().mean()*100

Company Name         0.0
Job Title            0.0
Job Description      0.0
Job Location         0.0
Job Posting URL      0.0
Normalized Salary    0.0
dtype: float64

## Reading OpenAI API Key

In [4]:
# Read the OpenAI API key
openai.api_key = open("OPENAI_API_Key.txt", "r").read().strip()
os.environ['OPENAI_API_KEY'] = openai.api_key

## Stage 1: User Intent Clarification

This method initiates the conversation from the chatbot to the end user. It will extract all the required variables required for searching the right jobs and providing the recommendations through conversational memory.
It takes in the following variables:
1. Education Details: Highest Academic Qualification
2. Years of Experience: Years of working experience in the industry
3. Technical Skills: All the technical skills shown for the job expertise
4. Preferred Locations: Preferred job locations for working on-site
5. Expected Salary: Expected Salary in USD per annum

In [5]:
def initialize_conversation():
    '''
    Returns a list [{"role": "system", "content": system_message}]
    '''

    delimiter = "####"

    example_user_dict = {'Education Details': "Graduate",
                        'Years of Experience':"2",
                        'Technical Skills': "Python, Data Science, Machine Learning, Deep Learning",
                        'Preferred Locations': "Atlanta, Las Vegas, California",
                        'Expected Salary': "100000"}

    example_user_req = {'Education Details': "_",
                        'Years of Experience':"_",
                        'Technical Skills': "_",
                        'Preferred Locations': "_",
                        'Expected Salary': "_"}

    system_message = f"""
    You are an intelligent job search expert and your goal is to find the best jobs for a user.
    You need to ask relevant questions and understand the user profile by analysing the user's responses.
    You final objective is to fill the values for the different keys ('Education Details','Years of Experience','Technical Skills','Preferred Locations','Expected Salary') in the python dictionary and be confident of the values.
    These key value pairs define the user's profile.
    The python dictionary looks like this
    {{'Education Details': 'values','Years of Experience': 'values','Technical Skills': 'values','Preferred Locations': 'values','Expected Salary': 'values'}}
    The value for 'Years of Experience' should be a numerical value extracted from the user's response.
    The value for 'Education Details' should be 'Undergraduate', 'Graduate' or 'Post Graduate' based on the highest educational qualification, as stated by user.
    The value for 'Technical Skills' can be anything related to specific knowledge or abilities required to perform particular tasks related to a profession, as stated by user.
    The value for 'Preferred Locations' can be any specific cities, states, provinces, regions, or countries, as stated by user. If the user is comfortable with anything or does not have preference, this field can be ignored.
    The value for 'Expected Salary' should be a numerical value extracted from the user's response. If the user is comfortable with anything or does not have preference, this field can be ignored.
    All the values in the example dictionary are only representative values.
    {delimiter}
    Here are some instructions around the values for the different keys. If you do not follow this, you'll be heavily penalised:
    - The value for 'Years of Experience' should be a numerical value extracted from the user's response.
    - The value for 'Education Details' should be 'Undergraduate', 'Graduate' or 'Post Graduate' based on the highest educational qualification, as stated by user.
    - The value for 'Technical Skills' can be anything related to specific knowledge or abilities required to perform particular tasks related to a profession, as stated by user.
    - The value for 'Preferred Locations' can be any specific cities, states, provinces, regions, or countries, as stated by user. If the user is comfortable with anything or does not have preference, this field can be ignored.
    - The value for 'Expected Salary' should be a numerical value extracted from the user's response. If the user is comfortable with anything or does not have preference, this field can be ignored.
    - 'Years of Experience' value needs to be greater than or equal to 0. If the user says less than that, please mention that the experience given is not in valid range.
    - 'Expected Salary' value needs to be greater than or equal to 0. If the user says less than that, please mention that the experience given is not in valid range.
    - Do not randomly assign values to any of the keys.
    - The values need to be inferred from the user's response.
    {delimiter}

    To fill the dictionary, you need to have the following chain of thoughts:
    Follow the chain-of-thoughts below and only output the final updated python dictionary for the keys as described in {example_user_req}. \n
    {delimiter}
    Thought 1: Ask a question to understand the user's profile and requirements. \n
    If their primary kind of job role is unclear. Ask followup questions to understand their needs.
    You are trying to fill the values of all the keys {{'Education Details',Years of Experience','Technical Skills','Preferred Locations','Expected Salary'}} in the python dictionary by understanding the user requirements.
    Identify the keys for which you can fill the values confidently using the understanding. \n
    Remember the instructions around the values for the different keys.
    If the necessary information has been extracted, only then proceed to the next step. \n
    Otherwise, rephrase the question to capture their profile clearly. \n

    {delimiter}
    Thought 2: Now, you are trying to fill the values for the rest of the keys which you couldn't in the previous step.
    Remember the instructions around the values for the different keys.
    Ask questions you might have for all the keys to strengthen your understanding of the user's profile.
    If yes, move to the next Thought. If no, ask question on the keys whose values you are unsure of. \n
    It is a good practice to ask question with a sound logic as opposed to directly citing the key you want to understand value for.
    {delimiter}

    {delimiter}
    Thought 3: Check if you have correctly updated the values for the different keys in the python dictionary.
    If you are not confident about any of the values, ask clarifying questions.
    {delimiter}

    {delimiter}
    Here is a sample conversation between the user and assistant:
    User: "Hi, I am a Computer science graduate."
    Assistant: "Great! As a Computer Science graduate, you are likely looking for an entry-level job. Hence, the job should be related to software engineering and development. Can you provide me with your length of working experience and skills?
    User: I am 3.2 years experienced in Software Development.
    Assistant: Thanks for sharing your work experience. May I know what kind of skills you have? Understanding the specific type of jobs will help me tailor my recommendations accordingly. Let me know if my understanding is correct until now."
    User: "I am primarily looking for Senior Software Development job roles."
    Assistant: "Thank you for providing that information. Working in Senior Software Development involves working with Front-end, Back-end, Database Administration. Do you have prior project work experiences in any of these areas? Understanding your skill set and experience will help determine the right jobs looking for candidates like you."
    User: "Yes, I have some project experience in Front-end development. I have good skills in HTML, CSS, Javascript, ReactJS, SQL and UX/UI Designing."
    Assistant: "Thank you for the information. Front-end development will require good skills in HTML, CSS and Javascript. To ensure I have a complete understanding of your requirements, I have one more question: Do you have any preference for job location?"
    User: "Yes, I am looking to work in California or Texas."
    Assistant:"Could you kindly let me know your expected salary for the job? This will help me find options that fit within your salary range while meeting the specified conditions."
    User: "I am expecting 50000 USD per annum"
    Assistant: "{example_user_dict}"
    {delimiter}

    Start with a short welcome message and encourage the user to share their requirements.
    """
    conversation = [{"role": "system", "content": system_message}]
    # conversation = system_message
    return conversation

In [6]:
# Define a Chat Completions API call
# Retry up to 6 times with exponential backoff, starting at 1 second and maxing out at 20 seconds delay
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def get_chat_completions(input, json_format = False):
    MODEL = 'gpt-3.5-turbo'

    system_message_json_output = """<<. Return output in JSON format to the key output.>>"""

    # If the output is required to be in JSON format
    if json_format == True:
        # Append the input prompt to include JSON response as specified by OpenAI
        input[0]['content'] += system_message_json_output

        # JSON return type specified
        chat_completion_json = openai.chat.completions.create(
            model = MODEL,
            messages = input,
            response_format = { "type": "json_object"},
            seed = 1234)

        output = json.loads(chat_completion_json.choices[0].message.content)

    # No JSON return type specified
    else:
        chat_completion = openai.chat.completions.create(
            model = MODEL,
            messages = input,
            seed = 2345)

        output = chat_completion.choices[0].message.content

    return output

In [7]:
def moderation_check(user_input):
    # Call the OpenAI API to perform moderation on the user's input.
    response = openai.moderations.create(input=user_input)

    # Extract the moderation result from the API response.
    moderation_output = response.results[0].flagged
    # Check if the input was flagged by the moderation system.
    if response.results[0].flagged == True:
        # If flagged, return "Flagged"
        return "Flagged"
    else:
        # If not flagged, return "Not Flagged"
        return "Not Flagged"

In [8]:
def intent_confirmation_layer(response_assistant):

    delimiter = "####"

    prompt = f"""
    You are a senior evaluator who has an eye for detail.The input text will contain a user requirement captured through 5 keys.
    You are provided an input. You need to evaluate if the input text has the following keys:
    {{
    'Education Details': 'values',
    'Years of Experience':'number',
    'Technical Skills':'values',
    'Preferred Locations':'values',
    'Expected Salary':'number'
    }}
    The values for the keys can be anything which should be related to the key asked.
    The 'Education Details' key can take a value only from Undergraduate/Graduate/Post Graduate.
    The 'Years of Experience' key can take only a numerical value.
    The 'Technical Skills' key can take any value related to specific knowledge or abilities required to perform particular tasks related to a profession.
    The 'Preferred Locations' key can take only geological location values of cities, states, regions, provinces, country or should be initialized to 'All' if the user is comfortable with any location.
    The 'Expected Salary' key can take only a numerical value or should be initialized to '0' if the user is comfortable with any salary.
    Next you need to evaluate if the keys have the the values filled correctly.
    Only output a one-word string in JSON format at the key 'result' - Yes/No.
    Thought 1 - Output a string 'Yes' if the values are correctly filled for all keys, otherwise output 'No'.
    Thought 2 - If the answer is No, mention the reason in the key 'reason'.
    Thought 3 - Think carefully before the answering.
    """

    messages=[{"role": "system", "content":prompt },
              {"role": "user", "content":f"""Here is the input: {response_assistant}""" }]

    response = openai.chat.completions.create(
                                    model="gpt-3.5-turbo",
                                    messages = messages,
                                    response_format={ "type": "json_object" },
                                    seed = 1234
                                    # n = 5
                                    )

    json_output = json.loads(response.choices[0].message.content)

    return json_output

In [9]:
def dictionary_present(response):
    delimiter = "####"

    user_req = {'Education Details': "Graduate",
                'Years of Experience':"2",
                'Technical Skills': "Python, Data Science, Machine Learning, Deep Learning",
                'Preferred Locations': "Atlanta, Las Vegas, California",
                'Expected Salary': "100000"}

    prompt = f"""You are a python expert. You are provided an input.
            You have to check if there is a python dictionary present in the string.
            It will have the following format {user_req}.
            Your task is to just extract the relevant values from the input and return only the python dictionary in JSON format.
            The output should match the format as {user_req}.

            {delimiter}
            Make sure that the value of budget is also present in the user input. ###
            The output should contain the exact keys and values as present in the input.
            Ensure the keys and values are in the given format:
            {{
            'Education Details': 'Undergraduate/Graduate/Post Graduate ',
            'Years of Experience':'numerical value',
            'Technical Skills':'any technical knowledge or skill for profession',
            'Preferred Locations':'any geological location',
            'Expected Salary':'numerical value'
            }}
            Here are some sample input output pairs for better understanding:
            {delimiter}
            input 1: - Education Details: B.Tech - Years of Experience: 3 years - Technical Skills: Java, .NET, Springboot, C#, AWS - Preferred Locations: New York, Washington D.C. - Expected Salary: 120000 USD
            output 1: {{'Education Details': 'Graduate', 'Years of Experience': '3', 'Technical Skills': 'Java, .NET, Springboot, C#, AWS', 'Preferred Locations': 'New York, Washington D.C.', 'Expected Salary': '120000'}}

            input 2: {{'Education Details': 'Diploma', 'Years of Experience': '0', 'Technical Skills': 'Java, HTML, CSS, Javascript, ReactJS, Angular, NodeJS', 'Preferred Locations': 'Open for all', 'Expected Salary': 'Anything'}}
            output 2: {{'Education Details': 'Undergraduate', 'Years of Experience': '0', 'Technical Skills': 'Java, HTML, CSS, Javascript, ReactJS, Angular, NodeJS', 'Preferred Locations': 'All', 'Expected Salary': '0'}}

            input 3: Here is your user profile 'Education Details': 'M.Tech','Years of Experience': '10.5','Technical Skills': 'Python, Data Cleaning and Preprocessing, Regression and Classification, Decision Trees & Random Forests, Neural Networks, NLP and Generative AI, AWS, Model deployment, CI/CD pipeline','Preferred Location': 'Work from home','Expected Salary': '2400000 USD'
            output 3: {{'Education Details': 'Post Graduate','Years of Experience': '10.5','Technical Skills': 'Python, Data Cleaning, Data Preprocessing, Regression and Classification, Decision Trees & Random Forests, Neural Networks, NLP and Generative AI, AWS, Model deployment, CI/CD pipeline, Preferred Location': 'Remote','Expected Salary': '2400000'}}
            {delimiter}
            """
    messages = [{"role": "system", "content":prompt },
                {"role": "user", "content":f"""Here is the user input: {response}""" }]

    confirmation = get_chat_completions(messages, json_format = True)

    return confirmation

In [10]:
def product_map_layer(job_description):
    delimiter = "#####"

    job_spec = {
        "Education Details":"(Type of the highest academic qualification - Undergraduate/Graduate/Post Graduate)",
        "Years of Experience":"(Number of years as working professional experience)",
        "Technical Skills":"(Types of skills and abilities qualified with expertise in a profession)"
    }

    degree_levels = {'Undergraduate','Graduate','Post Graduate'}

    prompt=f"""
    You are a Job Specifications Matcher whose duty is to extract the key features of job opening and associate them as per their requirements.
    To analyze each job, perform the following steps:
    Step 1: Extract the highest academic qualification required for the job from the description {job_description}
    Step 2: Extract the range given for experience required for the job from the description {job_description} as (num1, num2). If the experience is given in any number+, then extract it as (number, 45) 
    Step 3: Extract the technical skills required for the job from the description {job_description}
    Step 4: Store the extracted features in {job_spec} \
    Step 5: Classify 'Education Details' into {degree_levels} based on the following rules: \
    {delimiter}
    Education Details:
    - Undergraduate: <<< if academic qualification like bachelors, graduate is missing or contains only upto school or diploma level >>> , \n
    - Graduate: <<< if academic qualification is bachelors, graduate level >>> , \n
    - Post Graduate: <<< if academic qualification is masters, post graduate, doctorate level >>> , \n

    {delimiter}
    Here is input output pair for few-shot learning:
    input 1: "We are looking for a HR Generalist with 2-5 years of experience. The candidate should possess BBA and demonstrate strong skills in Onboarding, Recruitment, HRIS, Employee Files, Benefits, Payroll, Employee Relations. This role involves working on innovative projects in HR domain."
    output 1: {{'Education Details': 'Graduate','Years of Experience':'(2, 5)','Technical Skills':'Onboarding, Recruitment, HRIS, Employee Files, Benefits, Payroll, Employee Relations'}}

    input 2: "We are looking for a Director of Marketing with 8+ years of experience. The candidate should possess MBA and demonstrate strong skills in Email Marketing, Social Media, Content Scheduling, Copywriting, Brand Strategy, Paid Ads, Analytics, Campaign Management, Market Research, Budgeting, Agency Oversight, Executive Reporting. This role involves working on innovative projects in Marketing domain."
    output 2: {{'Education Details': 'Post Graduate','Years of Experience':'(8, 45)','Technical Skills':'Email Marketing, Social Media, Content Scheduling, Copywriting, Brand Strategy, Paid Ads, Analytics, Campaign Management, Market Research, Budgeting, Agency Oversight, Executive Reporting'}}
    
    input 3: "We are looking for a Medical Records Technician with 0-4 years of experience. The candidate should possess a degree in Diploma and demonstrate strong skills in Medical Records, EHR, Phlebotomy, Diagnostics, CPR. This role involves working on innovative projects in Healthcare domain."
    output 3: {{'Education Details': 'Undergraduate','Years of Experience':'(0, 4)','Technical Skills':'Medical Records, EHR, Phlebotomy, Diagnostics, CPR'}} 
    """
    input = f"""Follow the above instructions step-by-step and output the dictionary in JSON format {job_spec} for the following job {job_description}."""
    #see that we are using the Completion endpoint and not the Chatcompletion endpoint
    messages=[{"role": "system", "content":prompt },{"role": "user","content":input}]

    response = get_chat_completions(messages, json_format = True)

    return response

## Stage 2: Job Matching Engine

In [11]:
## Creating a new column 'Score' in the filtered DataFrame and initializing it to 0
filtered_jobs = job_df.copy()
## Create a new column "job_feature" that contains the dictionary of the product features
filtered_jobs['Job Features'] = filtered_jobs['Job Description'].apply(lambda x: product_map_layer(x))

In [20]:
def compare_jobs_with_user(user_profile, top_n=5):
    scores = []
    filtered2_jobs = filtered_jobs.copy()
    
    ## Experience match
    filtered2_jobs = filtered2_jobs[filtered2_jobs['Job Features'].apply(lambda x: ast.literal_eval(x['Years of Experience'])[1] >= float(user_profile['Years of Experience']))]
    filtered2_jobs = filtered2_jobs[filtered2_jobs['Job Features'].apply(lambda x: ast.literal_eval(x['Years of Experience'])[0] <= float(user_profile['Years of Experience']))]
    
    ## Education Details match
    user_degree = user_profile['Education Details']
    mappings = {'Undergraduate': 0, 'Graduate': 1, 'Post Graduate': 2}
    user_mapping = mappings[user_degree]
    filtered2_jobs = filtered2_jobs[filtered2_jobs['Job Features'].apply(lambda x: mappings[x['Education Details']] <= user_mapping)]

    for index, row in filtered2_jobs.iterrows():
        score = 0

        ## Skills match
        user_skills = {x.strip().lower() for x in user_profile['Technical Skills'].split(',') if x.strip()}
        job_skills = {x.strip().lower() for x in row['Job Features']['Technical Skills'].split(',') if x.strip()}
        score += len(user_skills & job_skills)

        ## Location match
        user_locations = {x.strip().lower() for x in user_profile['Preferred Locations'].split(',') if x.strip()}
        job_location = {x.strip().lower() for x in row['Job Location'].split(',') if x.strip()}
        if user_locations == {'all'} or not user_locations:
            score += 1
        else:
            score += len(user_locations & job_location)

        ## Salary match
        try:
            expected_salary = float(user_profile['Expected Salary'])
            normalized_salary = float(row['Normalized Salary'])
            if expected_salary <= normalized_salary:
                score += 1
        except:
            pass

        scores.append(score)
    
    filtered2_jobs['Score'] = scores
    top_jobs = filtered2_jobs.sort_values(by=['Score', 'Normalized Salary'], ascending=[False, False]).head(top_n)
    top_jobs_json = top_jobs.to_json(orient='records')

    return top_jobs_json

## Stage 3: Jobs Recommendation Engine

In [13]:
def recommendation_validation(job_recommendation):
    data = json.loads(job_recommendation)
    data1 = []
    for i in range(len(data)):
        if data[i]['Score'] >= 1:
            data1.append(data[i])

    return data1

In [15]:
def initialize_conv_reco(jobs):
    system_message = f"""
    You are an intelligent job requirements and skill matching expert and you are tasked with the objective to \
    solve the user queries about any job from the catalogue in the user message \
    You should keep the user profile in mind while answering the questions.\
    You should list out all 5 job recommendations.\

    Start with a brief summary of each job in the following format, in decreasing order of salary of jobs:
    1. <Job Title Name> at <Company Name>, <Job Location> with annual salary of <Normalized Salary in USD>. Apply here if you are interested: <Job Posting URL>
    2. <Job Title Name> at <Company Name>, <Job Location> with annual salary of <Normalized Salary in USD>. Apply here if you are interested: <Job Posting URL>
    3. <Job Title Name> at <Company Name>, <Job Location> with annual salary of <Normalized Salary in USD>. Apply here if you are interested: <Job Posting URL>
    4. <Job Title Name> at <Company Name>, <Job Location> with annual salary of <Normalized Salary in USD>. Apply here if you are interested: <Job Posting URL>
    5. <Job Title Name> at <Company Name>, <Job Location> with annual salary of <Normalized Salary in USD>. Apply here if you are interested: <Job Posting URL>

    """
    user_message = f""" These are the user's jobs: {jobs}"""
    conversation = [{"role": "system", "content": system_message },
                    {"role":"user","content":user_message}]
 
    return conversation

## Master Chatbot Controller

In [16]:
def dialogue_mgmt_system():
    conversation = initialize_conversation()
    introduction = get_chat_completions(conversation)
    display(introduction + '\n')

    top_5_jobs = None
    user_input = ''

    while user_input.lower() != "exit":
        user_input = input("")

        # Moderation check for user input
        if moderation_check(user_input) == 'Flagged':
            display("Sorry, this message has been flagged. Please restart your conversation.")
            break

        if top_5_jobs is None:
            # Append user input and generate assistant reply
            conversation.append({"role": "user", "content": user_input})
            response_assistant = get_chat_completions(conversation)

            # Moderation check for assistant response
            if moderation_check(response_assistant) == 'Flagged':
                display("Sorry, this message has been flagged. Please restart your conversation.")
                break

            # Intent Confirmation
            confirmation = intent_confirmation_layer(response_assistant)
            result = confirmation.get('result', '').strip().lower()

            print("Intent Confirmation Yes/No:", result.capitalize())

            if result == "yes":
                # Proceed to job recommendation
                print("\n" + response_assistant + "\n")
                print('\nVariables extracted!\n')

                # Parse structured dictionary from assistant's reply
                user_profile = dictionary_present(response_assistant)

                # Compare user profile with jobs
                print("Thank you for providing all the information. Kindly wait, while I fetch the jobs: \n")
                top_5_jobs = compare_jobs_with_user(user_profile, 5)

                if not top_5_jobs or len(json.loads(top_5_jobs)) == 0:
                    print("Sorry, no jobs matched your current preferences. You may want to relax your constraints.")
                    break

                print("Top 5 recommended jobs for you are:\n", top_5_jobs)

                validated_reco = recommendation_validation(top_5_jobs)

                conversation_reco = initialize_conv_reco(validated_reco)
                conversation_reco.append({"role": "user", "content": "This is my user profile: " + str(user_profile)})

                recommendation = get_chat_completions(conversation_reco)

                if moderation_check(recommendation) == 'Flagged':
                    display("Sorry, this message has been flagged. Please restart your conversation.")
                    break

                conversation_reco.append({"role": "assistant", "content": str(recommendation)})
                print("\n" + str(recommendation) + '\n')

            else:
                # Only append assistant message to conversation if incomplete
                reason = confirmation.get('reason', 'Some required information is missing or invalid.')
                print("\n" + response_assistant + "\n")
                print("Missing or incorrect information: ", reason)
                print("Please provide or correct the above info to proceed.\n")

        else:
            # Post-recommendation follow-up conversation
            conversation_reco.append({"role": "user", "content": user_input})
            response_asst_reco = get_chat_completions(conversation_reco)

            if moderation_check(response_asst_reco) == 'Flagged':
                print("Sorry, this message has been flagged. Please restart your conversation.")
                break

            print('\n' + response_asst_reco + '\n')
            conversation_reco.append({"role": "assistant", "content": response_asst_reco})

In [17]:
debug_response = {'Education Details': 'Graduate', 
                  'Years of Experience': '4.5', 
                  'Technical Skills': 'Java, Kubernetes, Git, .NET, Springboot, ReactJS, Javascript', 
                  'Preferred Locations': 'Austin, New York, Atlanta, Denver, Dallas', 
                  'Expected Salary': '110000'}

In [21]:
compare_jobs_with_user(debug_response, 5)

'[{"Company Name":"Google","Job Title":"Software Engineer","Job Description":"We are looking for a Software Engineer with 2-6 years of experience. The candidate should possess Bachelors in Computer Science and demonstrate strong skills in HTML, CSS, JavaScript, Git, Java, Spring Boot, SQL, REST APIs. This role involves working on innovative projects in Technology domain.","Job Location":"Omaha, NE","Job Posting URL":"https:\\/\\/www.linkedin.com\\/jobs\\/view\\/940525\\/?trk=jobs_biz_prem_srch","Normalized Salary":140000,"Job Features":{"Education Details":"Graduate","Years of Experience":"(2, 6)","Technical Skills":"HTML, CSS, JavaScript, Git, Java, Spring Boot, SQL, REST APIs"},"Score":4},{"Company Name":"Amazon","Job Title":"Software Engineer","Job Description":"We are looking for a Software Engineer with 2-6 years of experience. The candidate should possess Bachelors in Computer Science and demonstrate strong skills in HTML, CSS, JavaScript, Git, Java, Spring Boot, SQL, REST APIs. 

## Chatbot Evaluation

In [22]:
dialogue_mgmt_system()

"Hello! It's great to have you here. Please share your profile and job preferences so that I can assist you in finding the best job opportunities.\n"

 Hi, I'm a B.Tech Graduate in Computer science


Intent Confirmation Yes/No: No

Great! As a B.Tech graduate in Computer Science, you might be looking for job roles related to software engineering and development. Can you please provide me with details about your work experience and technical skills to better understand your job requirements?

Missing or incorrect information:  Missing key information
Please provide or correct the above info to proceed.



 I have 4.5 years of experience as Full stack developer. I'm skilled in Java, Kubernetes, Git, .NET, Springboot, ReactJS, Javascript.


Intent Confirmation Yes/No: No

Great to know about your background! As a B.Tech Graduate in Computer Science with 4.5 years of experience as a Full stack developer, your job roles are likely related to software engineering and development. Your skills in Java, Kubernetes, Git, .NET, Springboot, ReactJS, and Javascript indicate that you are proficient in both front-end and back-end technologies.

To better understand your requirements, could you please let me know if you have any specific preference for job locations?

Missing or incorrect information:  Education Details key is missing
Please provide or correct the above info to proceed.



...


Certainly! Here are 3 more job recommendations for you based on your user profile:

6. **Software Engineer at Microsoft, Austin, TX** with an annual salary of $120,000. The role requires a Bachelor's in Computer Science and 4 years of experience. Key skills include Java, Kubernetes, Git, .NET, Springboot, ReactJS, and Javascript. Apply here if you are interested: [Job Posting URL](example.com)

7. **Software Developer at Facebook, New York, NY** with an annual salary of $118,000. Seeking a candidate with a Bachelor's in Computer Science and 4 years of experience. Strong skills in Java, Kubernetes, Git, .NET, Springboot, ReactJS, and Javascript are required. Apply here if you are interested: [Job Posting URL](example.com)

8. **Full Stack Developer at Netflix, Atlanta, GA** with an annual salary of $115,000. The ideal candidate should have a Bachelor's in Computer Science and 4 years of experience. Proficiency in Java, Kubernetes, Git, .NET, Springboot, ReactJS, and Javascript is essen

 That's great, thank you so much! 



You're welcome! If you have any more questions or need further assistance in the future, feel free to ask. Good luck with your job search!



 exit



Goodbye! Feel free to return anytime if you need help. Have a great day!

